# Human/AI Speech Classfier

Discription:
This notebook is using Torch ML Liberary to import and train the HuBERT pre-traiend model. It uses librosa to load the wav, and we've also used Wav2Vec to extract the features.




##Import all the needed libraries



In [ ]:
!pip install transformers datasets
from google.colab import drive
import torch
import os
import pandas as pd
from pandas.core.frame import DataFrame
import IPython.display as ipd
from datasets import Dataset
import librosa
from transformers import HubertForSequenceClassification
from transformers import Wav2Vec2FeatureExtractor
from torch.optim import AdamW
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cuda_id = 0
drive.mount('/content/drive/', force_remount=True)
device = torch.device("cuda:%s" % cuda_id if torch.cuda.is_available() else "cpu")
device_name = torch.cuda.get_device_name(cuda_id) if torch.cuda.is_available() else "cpu"

Mounted at /content/drive/
We are using the device cpu - cpu


##Data prepration

In [ ]:
D_set = "/content/drive/MyDrive/voice_project/dataset/"
D_set2 = "/content/drive/MyDrive/voice_project/converted_dataset"
dir_list = os.listdir(D_set)
dir_list2 = os.listdir(D_set2)
for i in dir_list:
  dir_list[dir_list.index(i)] = 'dataset/'+i
for i in dir_list2:
 # print(i)
  dir_list.append('converted_dataset/'+i)
speaker = []
path = []

for i in dir_list:
  speaker.append(i[0])
  path.append('/' + i)

print(speaker)
D_set_df = pd.DataFrame(speaker)
D_set_df = D_set_df.replace({ 'd':'Human', 'c':'AI'})
label =  D_set_df.replace({ 'Human':1, 'AI':2})
D_set_df = pd.concat([D_set_df, label],axis=1)
D_set_df.columns = ['speaker','label']
D_set_df = pd.concat([D_set_df,pd.DataFrame(path, columns = ['path'])],axis=1)
D_set_df.speaker.value_counts()

['d', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd',

Human    14514
AI        2492
Name: emotion, dtype: int64

In [ ]:
D_set_df

,emotion,source,path
0,Human,SADA,/dataset/c_67.78999999999996_68.88999999999999...
1,Human,SADA,/dataset/c_67.80999999999972_68.67999999999984...
2,Human,SADA,/dataset/c_67.83_69.446k_v2ms_SBA_11_segmented...
3,Human,SADA,/dataset/c_67.84_69.056k_v2ms_SBA_669_segmente...
4,Human,SADA,/dataset/c_67.88000000000011_76.20000000000027...
...,...,...,...
17001,AI,SADA,/converted_dataset/c_ARA NORM 1415.wav
17002,AI,SADA,/converted_dataset/c_ARA NORM 1416.wav
17003,AI,SADA,/converted_dataset/c_ARA NORM 1417.wav
17004,AI,SADA,/converted_dataset/c_ARA NORM 1419.wav


Split into training data and test data.

In [ ]:
# D_set_df = D_set_df.sample(frac=0.1)
train_df = D_set_df.sample(frac=0.8)
test_df = D_set_df.drop(train_df.index)

In [ ]:
train_df

,emotion,source,path
3392,Human,SADA,/dataset/c_461.69_462.516k_v2ms_SBA_194_segmen...
9757,Human,SADA,/dataset/c_228.8_230.596k_v_SBA2_2023_0.wav
3787,Human,SADA,/dataset/c_48.13998118488168_53.11000000000036...
5361,Human,SADA,/dataset/c_367.36_370.36k_SBA_128_0.wav
14309,Human,SADA,/dataset/ARA NORM 1132.wav
...,...,...,...
11654,Human,SADA,/dataset/c_161.89_167.50231007453096k_SBA_115_...
11930,Human,SADA,/dataset/c_169.84999999999968_172.869999999999...
6175,Human,SADA,/dataset/c_318.34_324.696k_SBA_67_0.wav
2622,Human,SADA,/dataset/c_529.3000000000002_538.0500000000002...


In [ ]:
train_df["speaker"].value_counts()

Human    11652
AI        1953
Name: emotion, dtype: int64

##Testing the loaded dataset

In [ ]:
main_dir = '/content/drive/MyDrive/voice_project/'
print(train_df.iloc[0])
ipd.Audio(data=main_dir+train_df.iloc[id]["path"], autoplay=True, rate=16000)

emotion                                                Human
source                                                  SADA
path       /dataset/c_461.69_462.516k_v2ms_SBA_194_segmen...
Name: 3392, dtype: object


##Tokenization

In [ ]:
def map_to_array(Asample):
    speech, _ = librosa.load(main_dir+Asample["path"], sr=16000, mono=True)
    Asample["speech"] = speech
    return Asample

train_data = Dataset.from_pandas(train_df).map(map_to_array)
test_data = Dataset.from_pandas(test_df).map(map_to_array)
#using hubert with wav2vec to extract features
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-er")

train_encodings = feature_extractor(list(train_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt")
test_encodings = feature_extractor(list(test_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


Map:   0%|          | 0/13605 [00:00<?, ? examples/s]

Map:   0%|          | 0/3401 [00:00<?, ? examples/s]

##Turn data into a Dataset object

In [ ]:
class Speaker_dset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        pattern = {1:0, 2:0, 3:1, 4:3, 5:2}
        self.labels = [pattern[x] for x in labels]

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = Speaker_dset(train_encodings, list(train_data["labels"]))
test_dataset = Speaker_dset(test_encodings, list(test_data["labels"]))

##Loading the Model and Optimizer

In [ ]:
#using pre-triend model hubert form huggingface
model = HubertForSequenceClassification.from_pretrained("superb/hubert-large-superb-er")
model.to(device)
optim = AdamW(model.parameters(), lr=0.00001)

##Training

Train the model.

In [ ]:
epoch = 1
model.train()
def predict(outputs):
    probabilities = torch.softmax(outputs["logits"], dim=1)
    predictions = torch.argmax(probabilities, dim=1)
    return predictions

train_loss = list()
train_accuracies = list()
for epoch_i in range(epoch):
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    correct = 0
    count = 0
    epoch_loss = list()
    for batch in pbar:
        optim.zero_grad()
        input_ids = batch['input_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optim.step()
        predictions = predict(outputs)
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count
        epoch_loss.append(loss.item())
    train_loss += epoch_loss
    train_accuracies.append(accuracy)

In [ ]:
plt.plot(train_loss, label="Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Iteration vs Training Loss")
plt.legend()
plt.show()

In [ ]:
acc_X = np.arange(len(train_accuracies))+1
plt.plot(acc_X, train_accuracies,"-", label="Training Accuracy")
plt.xticks(acc_X)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Epoch vs Training Accuracy")
plt.legend()
plt.show()

NameError: ignored